In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [7]:
import numpy as np
from copy import deepcopy

dirs=[ (0,1), (-1,0), (0,-1), (1,0)]

class PathState:
    def __init__(self,loc, steps, came_from=None ):
        self.loc = loc
        self.steps = steps
        self.came_from=came_from

    def give_neighbors(self,score_map):
        neighs=[]
        nY,nX = len(score_map),len(score_map[0])
        y,x = self.loc

        for direction in dirs:
            ny,nx = direction

            if y+ny ==-1 or x+nx==-1 or x+nx==nX or y+ny==nY:
                continue
                
            if score_map[y+ny][x+nx] != '#':
                neighs.append(  PathState((y+ny,x+nx), 
                                    self.steps+1, 
                                    came_from=self,
                                )
                             )
        return neighs
                   
    def __str__(self):
        return f'({self.loc[0]},{self.loc[1]})'
        
    def __repr__(self):
        return f'({self.loc[0]},{self.loc[1]}) with {self.steps}'
    
    def __lt__(self,other):
        return self.steps < other.steps
        
    def cheat_comparison(self, other):
        path_diff = abs(self.steps - other.steps)
        manhatten = sum( [ abs(self.loc[x] - other.loc[x]) for x in range(2)])
        
        return manhatten, path_diff

    

In [8]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, PathState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: PathState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> PathState:
        return heapq.heappop(self.elements)[1]

In [75]:

def initial_map_read(score_map, start):
     # print out what we find
    frontier = PriorityQueue()
    s= PathState( start, 0 )
    frontier.put( s , priority=0)
    
    seen= {str(s):0}
    
    while not frontier.empty() :
        cur = frontier.get()
        
        y,x = cur.loc
        if score_map[y][x] == "E":
            return cur
            
        neighs = cur.give_neighbors(score_map)
               
        for n_loc in neighs:
            key= str(n_loc)
            if key not in seen.keys() or seen[key] > n_loc.steps:
                seen[key]=n_loc.steps
                frontier.put(n_loc, n_loc.steps)
                
    return None

def create_path_array(last_path_point):
    path_array =[]
    cur = last_path_point
    while cur.came_from != None :
        path_array.append(cur) 
        cur = cur.came_from
        
    path_array.append(cur)

    return path_array

def calculate_cheats(path, cheat_steps=2, gain_threshold=0):
    ncheats=0
    path.reverse()
    for i,start in enumerate(path[:-1]):
        for end in path[i+1:]:
            manhatten, path_diff = start.cheat_comparison(end)
            if manhatten <= cheat_steps and  ( path_diff - manhatten) > gain_threshold :
                ncheats+=1
    return ncheats
            

In [79]:
def day_20(path='input_data/test_20.txt', cheats=2):
    
    with open(path,'r') as f:
        score_map=[]
        start=None
        for y,line in enumerate(f.readlines()):
            line=line.strip()
            if 'S' in line:
                x = line.index('S')
                start= (y,x)
            score_map.append(list(line))

    last_loc = initial_map_read(score_map, start)
    path_array = create_path_array(last_loc)

    threshold = 100-1 if 'day_20' in path else 0 
    gains = calculate_cheats(path_array,cheat_steps=cheats,gain_threshold=threshold)
    return gains

In [77]:
%%time
part_one()

CPU times: user 3.84 ms, sys: 1.41 ms, total: 5.25 ms
Wall time: 4.67 ms


44

In [80]:
%%time
day_20('input_data/day_20.txt')

CPU times: user 11.7 s, sys: 75.1 ms, total: 11.8 s
Wall time: 11.8 s


1395

# Part 2

In [81]:
%%time
day_20('input_data/day_20.txt',20 )

CPU times: user 11.9 s, sys: 31.3 ms, total: 11.9 s
Wall time: 11.9 s


993178